In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser1"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Project Two')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className = 'row',
             style={'display': 'flex'},
                children = [
                    html.Button(id = 'submit-button-one', n_clicks = 0, children = 'Water Rescue'),
                    html.Button(id = 'submit-button-two', n_clicks = 0, children = 'Mountain or Wilderness Rescue')
                ]),
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        editable = False,
        filter_action = "native",
        sort_action = "native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = True,
        row_deletable = False,
        selected_columns=[],
        selected_rows=[],
        page_action = "native",
        page_current=0,
        page_size = 10,
        
    ),
    html.Br(),
     html.Hr(),
     html.Div(
            id='map-id',
            className='col s12 m6',)
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-interactivity', "data"),
    [Input('submit-button-one', 'n_clicks'),Input('submit-button-two', 'n_clicks'),
    ])

def on_click(bt1,bt2):
    #start case
    if (int(bt1) == 0 and int(bt2) == 0):
        df = pd.DataFrame.from_records(shelter.readAll({}))
    #use higher number of button clicks to determine filter type
    elif (int(bt1) > int(bt2)):
        df = pd.DataFrame(list(shelter.readAll({"animal_type":"Cat"})))
    elif (int(bt2) > int(bt1)):
        df = pd.DataFrame(list(shelter.readAll({"animal_type":"Dog"})))
    
    return df.to_dict('records')


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    return [
        d1.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            d1.TileLayer(id="base-layer-id"),
            #marker
            d1.Marker(position=[30.75,-97.48], children=[
                d1.Tooltip(dff.iloc[0,4]),
                d1.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

app

OperationFailure: Authentication failed.